In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

In [ ]:
os.environ["OPEN_AI_KEY"] = os.getenv("OPEN_AI_KEY")
os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_API_VERSION")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [17]:
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model="gpt-4o", api_key=os.getenv("OPEN_AI_KEY"), api_version=os.getenv("OPENAI_API_VERSION"))


In [4]:
result = llm.invoke("What is Google")

print(result.content)

Google is a multinational technology company that specializes in internet-based services and products. It is widely known for its search engine, which is one of the most popular and widely used services in the world. Google offers a variety of other products and services, including:

1. **Google Search:** The company's flagship product, which enables users to find information on the web.
2. **YouTube:** A video-sharing platform where users can upload, view, and share videos.
3. **Gmail:** A free email service with various features, including spam filtering, email storage, and integration with other Google services.
4. **Google Maps:** A mapping service providing directions, real-time traffic updates, and various location-based services.
5. **Google Drive:** A cloud storage service that allows users to store files and access them from any device.
6. **Google Docs, Sheets, and Slides:** Online office productivity tools for creating documents, spreadsheets, and presentations.
7. **Android

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# prompt=ChatPromptTemplate.from_messages([
#     ("system", "you are customer support from doko moko you need to asnwer user queries"),
#     ("user", "{input}")
# ])
# prompt

output_parser  = JsonOutputParser()
# prompt = PromptTemplate(
#     template="Answer the user query. \n{format_instructions}\n{query}",
#     input_variables=["query"],
#     partial_variables={"format_instructions":{output_parser.get_format_instructions()}}
# )

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/RajuGangitla/nueral-backend")
loader


In [9]:
document = loader.load()

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitting = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents = text_splitting.split_documents(document)

In [ ]:
import os
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("OPEN_AI_KEY"),
        api_version="2023-05-15"
        )

AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000258DB7EDE20>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x00000258DB8687A0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='2023-05-15', openai_api_base=None, openai_api_type='azure', openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=2048, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True, azure_endpoint='https://zedblockdev.openai.azure.com', azure_ad_token=None, azure_ad_token_provider=None, azure_ad_async_token_provider=None, validat

In [12]:
from langchain_community.vectorstores import FAISS

vectorstore  = FAISS.from_documents(documents, embeddings)
vectorstore

In [13]:
query = "Raju"
result = vectorstore.similarity_search(query=query)

result[0].page_content

'Code\n\n\n\n\n\n\n\nIssues\n0\n\n\n\n\n\n\nPull requests\n0\n\n\n\n\n\n\nActions\n\n\n\n\n\n\n\nProjects\n0\n\n\n\n\n\n\nSecurity\n\n\n\n\n\n\n\nInsights\n\n\n\n \n\n \n\n\nAdditional navigation options\n\n\n \n\n\n\n\n\n\n\n\n\n\n          Code\n\n\n\n\n\n\n\n\n\n\n\n          Issues\n\n\n\n\n\n\n\n\n\n\n\n          Pull requests\n\n\n\n\n\n\n\n\n\n\n\n          Actions\n\n\n\n\n\n\n\n\n\n\n\n          Projects\n\n\n\n\n\n\n\n\n\n\n\n          Security\n\n\n\n\n\n\n\n\n\n\n\n          Insights\n\n\n\n\n\n\n \n\n\n\n\n\nRajuGangitla/nueral-backend'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the follwing questions from the provided context : 
    <content>
    {context}
    </content>
    """
)


In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


ValueError: Prompt must accept context as an input variable. Received prompt with input variables: []